### Importacion de las Librerías

In [13]:
import sys
from pathlib import Path
import sklearn
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr
from PIL import Image
import numpy as np
import os
import shutil

print("Python:", sys.version)
print("Pathlib: incluida en la stdlib")
print("scikit-learn:", sklearn.__version__)
print("TensorFlow:", tf.__version__)
print("Keras:", tf.keras.__version__)
print("Gradio:", gr.__version__)
print("Pillow:", Image.__version__)
print("NumPy:", np.__version__)
print("os: incluida en la stdlib")
print("shutil: incluida en la stdlib")


Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
Pathlib: incluida en la stdlib
scikit-learn: 1.8.0
TensorFlow: 2.20.0
Keras: 3.12.0
Gradio: 6.1.0
Pillow: 12.0.0
NumPy: 2.3.5
os: incluida en la stdlib
shutil: incluida en la stdlib


### Razas Etiquetadas en el Dataset

In [4]:
cat_breeds = [
    'Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
    'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue',
    'Siamese', 'Sphynx'
]

### Clasificacion de las imágenes

In [5]:
images_path = Path("data/raw/images")
all_images = list(images_path.glob("*.jpg"))

print(f"Total de imágenes encontradas: {len(all_images)}")

cats = []
not_cats = []

for img_path in all_images:
    breed = "_".join(img_path.stem.split("_")[:-1])
    if breed in cat_breeds:
        cats.append(img_path)
    else:
        not_cats.append(img_path)

print(f"Gatos: {len(cats)}")
print(f"No gatos: {len(not_cats)}")

Total de imágenes encontradas: 7390
Gatos: 2400
No gatos: 4990


### Particion de la data en entrenamiento y evaluación

In [6]:
cats_train, cats_val = train_test_split(cats, test_size=0.2, random_state=42)
not_cats_train, not_cats_val = train_test_split(not_cats, test_size=0.2, random_state=42)


### Copiado de la data
Copiamos las unidades de entrenamiento y evaluación en sus respectivas carpetas

In [7]:
def safe_copy_list_to_folder(file_list, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    else:
        # Vaciar carpeta para evitar duplicados
        for filename in os.listdir(dest_folder):
            file_path = os.path.join(dest_folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    for f in file_list:
        shutil.copy(str(f), dest_folder)


print("Copiando imágenes...")

safe_copy_list_to_folder(cats_train, "data/processed/train/cat")
safe_copy_list_to_folder(cats_val, "data/processed/val/cat")
safe_copy_list_to_folder(not_cats_train, "data/processed/train/not_cat")
safe_copy_list_to_folder(not_cats_val, "data/processed/val/not_cat")

print("Copia completada.")
print(f"Train -> Gatos: {len(cats_train)}, No gatos: {len(not_cats_train)}")
print(f"Val   -> Gatos: {len(cats_val)}, No gatos: {len(not_cats_val)}")



Copiando imágenes...
Copia completada.
Train -> Gatos: 1920, No gatos: 3992
Val   -> Gatos: 480, No gatos: 998


### Carga, normalización y generación de imágenes

In [8]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "data/processed/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=["not_cat", "cat"]
)

val_generator = val_datagen.flow_from_directory(
    "data/processed/val",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=["not_cat", "cat"]
)


Found 5912 images belonging to 2 classes.
Found 1478 images belonging to 2 classes.


### Modelo Tranfer Learning

In [9]:
def create_transfer_learning_model():
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet"
    )
    base.trainable = False

    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

model = create_transfer_learning_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Entrenamiento

In [10]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2)
    ]
)



Epoch 1/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 184s 968ms/step - accuracy: 0.9383 - loss: 0.1766 - val_accuracy: 0.9871 - val_loss: 0.0612 - learning_rate: 0.0010
Epoch 2/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.9755 - loss: 0.0749 - val_accuracy: 0.9892 - val_loss: 0.0413 - learning_rate: 0.0010
Epoch 3/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 183s 989ms/step - accuracy: 0.9795 - loss: 0.0630 - val_accuracy: 0.9892 - val_loss: 0.0373 - learning_rate: 0.0010
Epoch 4/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 184s 996ms/step - accuracy: 0.9819 - loss: 0.0561 - val_accuracy: 0.9912 - val_loss: 0.0318 - learning_rate: 0.0010
Epoch 5/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.9799 - loss: 0.0554 - val_accuracy: 0.9899 - val_loss: 0.0316 - learning_rate: 0.0010
Epoch 6/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.9833 - loss: 0.0498 - val_accuracy: 0.9912 - val_loss: 0.0289 - learning_rate: 0.0010
Epoch 7/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.9838 -

### Guardado del modelo

In [11]:
model.save("model/cat_classifier.keras")
model.save("model/cat_classifier.h5")
model.export("model/cat_classifier_savedmodel")
print("Modelos guardados.")


INFO:tensorflow:Assets written to: model/cat_classifier_savedmodel\assets


INFO:tensorflow:Assets written to: model/cat_classifier_savedmodel\assets


Saved artifact at 'model/cat_classifier_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2697860643344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063008144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063009296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2697860642384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2697860643728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2697860643920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063009872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063010640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063010256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063009488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2698063009680: 

### Interfaz Gráfica

In [ ]:
model = tf.keras.models.load_model("model/cat_classifier.keras")

def predict_cat(image):
    img = image.resize((224, 224))
    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)

    p = model.predict(arr)[0][0]

    if p > 0.5:
        return f" Es un GATO (Confianza: {p*100:.2f}%)"
    else:
        return f"NO es un gato (Confianza: {(1-p)*100:.2f}%)"

gr.Interface(
    fn=predict_cat,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Clasificador de Gatos",
    description="Sube una imagen para identificar si contiene un gato."
).launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
